# Greenplum Demo (Part 4)

This is Part 4 of Greenplum Demo, ***Table Storage Models***. 

- If you missed Part 1 (*Setup, Describe Input Dataset & Data Loading*) or wish to repeat, then click [here](AWS-GP-demo-1.ipynb).
- If you missed Part 2 (*Basic Table Functions*) or wish to repeat, then click [here](AWS-GP-demo-2.ipynb).
- If you missed Part 3 (*MPP Fundamentals and Partitioning*) or wish to repeat, then click [here](AWS-GP-demo-3.ipynb).

In [1]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

In [2]:
%reload_ext sql
%sql $CONNECTION_STRING

u'Connected: gpadmin@gpadmin'

In [3]:
query = !cat script/7-db-maintenance.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

Done.
Done.
Done.
1 rows affected.
Done.


[]

## 7. Table Storage Models

### 7.1. Comparing Greenplum Table Storage Models: Loading

Re-create the Amazon Reviews table, using 3 different table storage models, Heap table, Append-Optimized (AO)/Row-Oriented table with ZLib (Level 3) compression, and Append-Optimized (AO)/Column-Oriented table with ZLib (Level 3) compression, as shown below:

In [4]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-heap.sql
sqlfilecode2 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-ao-ro-zlib3.sql
sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-ao-co-zlib3.sql

display_html('\n'.join(sqlfilecode1), raw=True)
display_html('\n'.join(sqlfilecode2), raw=True)
display_html('\n'.join(sqlfilecode3), raw=True)

query1 = !cat script/7-1-amzn-reviews-heap.sql
query2 = !cat script/7-1-amzn-reviews-ao-ro-zlib3.sql
query3 = !cat script/7-1-amzn-reviews-ao-co-zlib3.sql

%sql $DB_USER@$DB_SERVER {''.join(query1)}
%sql $DB_USER@$DB_SERVER {''.join(query2)}
%sql $DB_USER@$DB_SERVER {''.join(query3)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_heap ; 

 CREATE TABLE demo . amzn_reviews_heap ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 DISTRIBUTED BY ( review_id );

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_ao_ro_zlib3 ; 

 CREATE TABLE demo . amzn_reviews_ao_ro_zlib3 ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 WITH ( appendonly = true , orientation = row , compresstype = zlib , compresslevel = 3 ) 
 DISTRIBUTED BY ( review_id );

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_ao_co_zlib3 ; 

 CREATE TABLE demo . amzn_reviews_ao_co_zlib3 ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 WITH ( appendonly = true , orientation = column , compresstype = zlib , compresslevel = 3 ) 
 DISTRIBUTED BY ( review_id );

Done.
Done.
Done.
Done.
Done.
Done.


[]

Load the input dataset to each using the `gpload` utility, and compare loading times.

In [5]:
!scp -i ~/.ssh/aws-gp.pem script/7-1-gpload-amzn-reviews-heap.yaml $DB_USER@$DB_SERVER:gpload-amzn-reviews-heap.yaml
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'gpload -d gpadmin -f /home/gpadmin/gpload-amzn-reviews-heap.yaml 2>&1 \
    | tee /home/gpadmin/gpload-amzn-reviews-heap.yaml.log'

7-1-gpload-amzn-reviews-heap.yaml             100%  379    49.0KB/s   00:00    
2019-09-04 14:54:41|INFO|gpload session started 2019-09-04 14:54:41
2019-09-04 14:54:41|INFO|no host supplied, defaulting to localhost
2019-09-04 14:54:41|INFO|started gpfdist -p 8000 -P 9000 -f "/home/gpadmin/data/amazon_reviews_us*.tsv.gz" -t 30 -m 1000000
2019-09-04 14:54:41|INFO|did not find an external table to reuse. creating ext_gpload_reusable_8ad99f26_cf1b_11e9_8dfe_063f90b01baa
2019-09-04 15:01:01|WARN|3714 bad rows
2019-09-04 15:01:01|WARN|Please use following query to access the detailed error
2019-09-04 15:01:01|WARN|select * from gp_read_error_log('ext_gpload_reusable_8ad99f26_cf1b_11e9_8dfe_063f90b01baa') where cmdtime > to_timestamp('1567605281.21')
2019-09-04 15:01:01|INFO|running time: 380.77 seconds
2019-09-04 15:01:01|INFO|rows Inserted          = 103145273
2019-09-04 15:01:01|INFO|rows Updated           = 0
2019-09-04 15:01:01|INFO|data formatting errors = 3714
2019-09-04 15:01:01|INFO|

In [6]:
!scp -i ~/.ssh/aws-gp.pem script/7-1-gpload-amzn-reviews-ao-ro-zlib3.yaml $DB_USER@$DB_SERVER:gpload-amzn-reviews-ao-ro-zlib3.yaml
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'gpload -d gpadmin -f /home/gpadmin/gpload-amzn-reviews-ao-ro-zlib3.yaml 2>&1 \
    | tee /home/gpadmin/gpload-amzn-reviews-ao-ro-zlib3.log'

7-1-gpload-amzn-reviews-ao-ro-zlib3.yaml      100%  385    52.0KB/s   00:00    
2019-09-04 15:01:02|INFO|gpload session started 2019-09-04 15:01:02
2019-09-04 15:01:02|INFO|no host supplied, defaulting to localhost
2019-09-04 15:01:02|INFO|started gpfdist -p 8000 -P 9000 -f "/home/gpadmin/data/amazon_reviews_us*.tsv.gz" -t 30 -m 1000000
2019-09-04 15:01:03|INFO|did not find an external table to reuse. creating ext_gpload_reusable_6e5ca05e_cf1c_11e9_8275_063f90b01baa
2019-09-04 15:07:15|WARN|3714 bad rows
2019-09-04 15:07:15|WARN|Please use following query to access the detailed error
2019-09-04 15:07:15|WARN|select * from gp_read_error_log('ext_gpload_reusable_6e5ca05e_cf1c_11e9_8275_063f90b01baa') where cmdtime > to_timestamp('1567605662.91')
2019-09-04 15:07:15|INFO|running time: 372.97 seconds
2019-09-04 15:07:15|INFO|rows Inserted          = 103145273
2019-09-04 15:07:15|INFO|rows Updated           = 0
2019-09-04 15:07:15|INFO|data formatting errors = 3714
2019-09-04 15:07:15|INFO|

In [7]:
!scp -i ~/.ssh/aws-gp.pem script/7-1-gpload-amzn-reviews-ao-co-zlib3.yaml $DB_USER@$DB_SERVER:gpload-amzn-reviews-ao-co-zlib3.yaml
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'gpload -d gpadmin -f /home/gpadmin/gpload-amzn-reviews-ao-co-zlib3.yaml 2>&1 \
    | tee /home/gpadmin/gpload-amzn-reviews-ao-co-zlib3.log'

7-1-gpload-amzn-reviews-ao-co-zlib3.yaml      100%  386    53.6KB/s   00:00    
2019-09-04 15:07:16|INFO|gpload session started 2019-09-04 15:07:16
2019-09-04 15:07:16|INFO|no host supplied, defaulting to localhost
2019-09-04 15:07:16|INFO|started gpfdist -p 8000 -P 9000 -f "/home/gpadmin/data/amazon_reviews_us*.tsv.gz" -t 30 -m 1000000
2019-09-04 15:07:16|INFO|did not find an external table to reuse. creating ext_gpload_reusable_4d2d2eca_cf1d_11e9_9c32_063f90b01baa
2019-09-04 15:13:29|WARN|3714 bad rows
2019-09-04 15:13:29|WARN|Please use following query to access the detailed error
2019-09-04 15:13:29|WARN|select * from gp_read_error_log('ext_gpload_reusable_4d2d2eca_cf1d_11e9_9c32_063f90b01baa') where cmdtime > to_timestamp('1567606036.73')
2019-09-04 15:13:29|INFO|running time: 372.76 seconds
2019-09-04 15:13:29|INFO|rows Inserted          = 103145273
2019-09-04 15:13:29|INFO|rows Updated           = 0
2019-09-04 15:13:29|INFO|data formatting errors = 3714
2019-09-04 15:13:29|INFO|

In [8]:
cmd = 'grep -e '"'"'running'"'"' /home/gpadmin/gpload-amzn-reviews*\
    | awk '"'"'BEGIN{FS=":"} {print $1, "finished in", $5}'"'"'' 
grep_output = !ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd | pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(grep_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 / home / gpadmin / gpload - amzn - reviews - ao - co - zlib3 . log finished in 372.76 seconds 
 / home / gpadmin / gpload - amzn - reviews - ao - ro - zlib3 . log finished in 372.97 seconds 
 / home / gpadmin / gpload - amzn - reviews - heap . yaml . log finished in 380.77 seconds

### 7.2. Comparing Greenplum Table Storage Models: Table Size and Disk Space Usage

In [9]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/7-2-table-size-comparison.sql
display_html('\n'.join(sqlfilecode), raw=True)
query = !cat script/7-2-table-size-comparison.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT pg_namespace . nspname AS schema , 
 pg_class . relname AS relation , 
 pg_size_pretty ( sotd . sotdsize :: BIGINT ) as tablesize , 
 pg_size_pretty ( sotd . sotdtoastsize :: BIGINT ) as toastsize , 
 pg_size_pretty ( sotd . sotdadditionalsize :: BIGINT ) as othersize , 
 pg_size_pretty ( sotaid . sotaidtablesize :: BIGINT ) as tabledisksize , 
 pg_size_pretty ( sotaid . sotaididxsize :: BIGINT ) as indexsize , 
 pg_size_pretty ( sotu . sotusize :: BIGINT ) as uncompressedsize , 
 ROUND (( 100.0 * ( 1 - ( sotaid . sotaidtablesize / sotu . sotusize ))) :: NUMERIC , 2 ) AS compressionpercentage 
 FROM pg_class 
 LEFT JOIN pg_stat_user_tables 
 ON pg_stat_user_tables . relid = pg_class . oid 
 LEFT JOIN gp_toolkit . gp_size_of_table_disk sotd 
 ON sotd . sotdoid = pg_class . oid 
 LEFT JOIN gp_toolkit . gp_size_of_table_and_indexes_disk sotaid 
 ON sotaid . sotaidoid = pg_class . oid 
 LEFT JOIN gp_toolkit . gp_size_of_table_uncompressed sotu 
 ON sotu . sotuoid = pg_class . oid 
 LEFT JOIN pg_namespace 
 ON pg_namespace . oid = pg_class . relnamespace 
 WHERE 
 pg_class . relkind = 'r' 
 AND relstorage != 'x' 
 AND pg_namespace . nspname NOT IN ( 'information_schema' , 'madlib' , 'pg_catalog' , 'gptext' ) 
 AND pg_class . relname NOT IN ( 'spatial_ref_sys' ) 
 ORDER BY 1 , 2

5 rows affected.


schema,relation,tablesize,toastsize,othersize,tabledisksize,indexsize,uncompressedsize,compressionpercentage
demo,amzn_reviews_ao_co_zlib3,24 GB,800 kB,1600 kB,24 GB,0 bytes,55 GB,56.14
demo,amzn_reviews_ao_ro_zlib3,28 GB,195 MB,1600 kB,28 GB,0 bytes,57 GB,50.57
demo,amzn_reviews_by_marketplace,59 GB,195 MB,0 bytes,60 GB,0 bytes,60 GB,0.00
demo,amzn_reviews_heap,59 GB,195 MB,0 bytes,60 GB,0 bytes,60 GB,0.00
demo,calendar,768 kB,0 bytes,0 bytes,768 kB,0 bytes,768 kB,0.00


### 7.3. Comparing Greenplum Table Storage Models: Query Performance

#### 7.3.1. Narrow (*Few columns of the table*) `SELECT`

In [10]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-narrow-select-heap.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-narrow-select-heap.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode2 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-narrow-select-ao-ro.sql
display_html('\n'.join(sqlfilecode2), raw=True)
cmd2 = !echo $(cat script/7-3-narrow-select-ao-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd2), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-narrow-select-ao-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-narrow-select-ao-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT product_id , 
 marketplace , 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_heap 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 2846.315 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT product_id , 
 marketplace , 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_ao_ro_zlib3 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 15181.699 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT product_id , 
 marketplace , 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_ao_co_zlib3 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 2990.255 ms

#### 7.3.2. Super Narrow (*1 column of the table*) `SELECT`

In [11]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-heap.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-super-narrow-select-heap.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode2 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-ao-ro.sql
display_html('\n'.join(sqlfilecode2), raw=True)
cmd2 = !echo $(cat script/7-3-super-narrow-select-ao-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd2), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-ao-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-super-narrow-select-ao-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT review_id 
 FROM demo . amzn_reviews_heap 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 2276.568 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT review_id 
 FROM demo . amzn_reviews_ao_ro_zlib3 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 15162.166 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT review_id 
 FROM demo . amzn_reviews_ao_co_zlib3 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 2570.328 ms

#### 7.3.3. Wide (*Most/Many columns of the table*) `SELECT`

In [12]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-wide-select-heap.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-wide-select-heap.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode2 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-wide-select-ao-ro.sql
display_html('\n'.join(sqlfilecode2), raw=True)
cmd2 = !echo $(cat script/7-3-wide-select-ao-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd2), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-wide-select-ao-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-wide-select-ao-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT marketplace , 
 customer_id , 
 product_id , 
 product_title , 
 product_parent , 
 product_category , 
 star_rating , 
 helpful_votes , 
 total_votes , 
 vine , 
 verified_purchase , 
 review_headline , 
 review_body 
 FROM 
 demo . amzn_reviews_heap 
 WHERE 
 DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY 
 RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 3403.284 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT marketplace , 
 customer_id , 
 product_id , 
 product_title , 
 product_parent , 
 product_category , 
 star_rating , 
 helpful_votes , 
 total_votes , 
 vine , 
 verified_purchase , 
 review_headline , 
 review_body 
 FROM 
 demo . amzn_reviews_ao_ro_zlib3 
 WHERE 
 DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY 
 RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 15540.724 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT marketplace , 
 customer_id , 
 product_id , 
 product_title , 
 product_parent , 
 product_category , 
 star_rating , 
 helpful_votes , 
 total_votes , 
 vine , 
 verified_purchase , 
 review_headline , 
 review_body 
 FROM 
 demo . amzn_reviews_ao_co_zlib3 
 WHERE 
 DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY 
 RANDOM () 
 LIMIT 1000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 13731.001 ms

#### 7.3.4. Aggregate/Window Functions over a limited number of columns

In [13]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-heap.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-aggr-select-heap.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode2 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-ao-ro.sql
display_html('\n'.join(sqlfilecode2), raw=True)
cmd2 = !echo $(cat script/7-3-aggr-select-ao-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd2), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-ao-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-aggr-select-ao-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Total runtime') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 marketplace , 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_heap 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 GROUP BY 2 , 3 , 4 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 2708.238 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 marketplace , 
 product_category , star_rating 
 FROM demo . amzn_reviews_ao_ro_zlib3 
 WHERE 
 DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 GROUP BY 2 , 3 , 4 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 15506.350 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 marketplace , 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_ao_co_zlib3 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 GROUP BY 2 , 3 , 4 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Total runtime : 2293.790 ms